In [1]:
#import the library Pandas
import pandas as pd

In [2]:
#Put the spam.csv dataset on Root Directory
#Read the dataset 
data = pd.read_csv("spam.csv", encoding = "latin-1")

In [3]:
# Changed the column names to be more descriptive.
data = data[['v1', 'v2']]
data = data.rename(columns = {'v1': 'label', 'v2': 'text'})

In [4]:
#Showing the data
data

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
# For the classification problem, we will only use case normalisation. 
# The rationale is that it will be hard to apply a stemmer or lemmatiser onto colloquial English 
# and that since the text messages are so short, removing stop words might not leave us 
# with much to work with.
def review_messages(msg):
    # converting messages to lowercase
    msg = msg.lower()
    return msg

In [6]:
# For reference, this function does case normalisation, removing stop words and stemming.
from nltk import stem
from nltk.corpus import stopwords
stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

def alternative_review_messages(msg):
    # converting messages to lowercase
    msg = msg.lower()
    # removing stopwords
    msg = [word for word in msg.split() if word not in stopwords]
    # using a stemmer
    msg = " ".join([stemmer.stem(word) for word in msg])
    return msg

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/aman/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# We apply the first function to normalise the text messages.
data['text'] = data['text'].apply(review_messages)

In [ ]:
# Vectorizing the Text
# Before training the vectorizer, we split our data into a training set and a testing set. 
# 10% of our data is allocated for testing.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size = 0.1, random_state = 1)
# training the vectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)

NameError: name 'data' is not defined

In [ ]:
# Building and Testing the Classifier
from sklearn import svm
svm = svm.SVC(C=1000)
svm.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
# Now, let’s test it.
from sklearn.metrics import confusion_matrix
X_test = vectorizer.transform(X_test)
y_pred = svm.predict(X_test)
print(confusion_matrix(y_test, y_pred))

[[490   0]
 [ 10  58]]


In [ ]:
# The results aren’t bad at all! We have no false positives and around 15% false negatives.
# Let’s test it against a few new examples.
def pred(msg):
    msg = vectorizer.transform([msg])
    prediction = svm.predict(msg)
    return prediction[0]

In [ ]:
data['text'] = data['text'].apply(pred)